<a href="https://colab.research.google.com/github/Anushree-B/Lie-detector/blob/main/lie_detector_sentence_analysis_(NLP).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lie detector using neural network

## Step 2 : Senetence Analysis

Based on the analysis by Victor Palacios https://github.com/Victor-Palacios/Deception_Detection_Capstone, this notebook dives into analyzing sentences for our project on detecting deception in political speech. We'll dissect 14400 labeled quotes (true, false, etc.) to identify linguistic patterns that might differentiate truth from lies.

Here's how we'll dissect these quotes:

- **Word Breakdown**: We'll calculate the total number of words to understand the overall conciseness or complexity of statements.
- **Part-of-Speech Distribution**: We'll analyze the frequency of verbs, adverbs, adjectives, and nouns. This can reveal patterns in how politicians express information (e.g., high noun count for factual claims, frequent adverbs for softening statements).
- **Sentiment Analysis**: We'll determine the emotional tone (positive, negative, neutral) of each quote. This can highlight potential biases or manipulative language.
- **Vocabulary Exploration**: We'll explore the specific words used, identifying keywords or phrases that might correlate with truthfulness.

Thus, based on this analysis, aim is to create a dataset contating all the above factors. This will be done using Natural language processing.

## Importing libraries

In [1]:
import numpy as np
import pandas as pd
import re
import nltk
from tqdm import tqdm

In [2]:
df = pd.read_csv("Data/politifact.csv")
df.head()

,Politician,Quote,Image URL
0,Melissa Agard,potato chips kitkat bars and viagra are not t...,true
1,Melissa Agard,since the state senate has only rejected...,true
2,Instagram posts,lego donates model mri kits to hospitals to he...,true
3,Brian Krassenstein,it has been u s policy for at least years...,true
4,Mike Oliverio,a few years back mitchell stadium in bluefie...,true


## adding character count, word count and word length of Quote

In [3]:
df['char_count'] = np.zeros(14400)
df['word_count'] = np.zeros(14400)
df['word_length'] = np.zeros(14400)

df['char_count'] = df['Quote'].apply(len)
df['word_count'] = df['Quote'].apply(lambda x: len(x.split()))
df['word_length'] = df['Quote'].apply(lambda x: np.mean([len(word) for word in x.split()]))

## FInding the number of adverbs, adjectives, nouns and verbs in Quote

In [5]:
nltk.download('averaged_perceptron_tagger') #used for tagging words with their parts of speech (POS)
nltk.download('wordnet')

adv_count = adj_count = noun_count = verb_count = det_count = np.zeros(14400)

for i in tqdm(range(0,14400)):
  words = df['Quote'][i].split()
  tagged_words = nltk.pos_tag(words)
  for word, tag in tagged_words:
    if tag.startswith('RB'):
      adv_count[i] += 1
    elif tag.startswith('JJ'):
      adj_count[i] += 1
    elif tag.startswith('NN'):
      noun_count[i] += 1
    elif tag.startswith('VB'):
      verb_count[i] += 1
    elif tag.startswith('DT'):
      det_count[i] += 1

df['adv_count'] = adv_count
df['adj_count'] = adj_count
df['noun_count'] = noun_count
df['verb_count'] = verb_count
df['det_count'] = det_count

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
100%|██████████| 14400/14400 [00:19<00:00, 740.86it/s]


In [7]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Download VADER lexicon (if not already downloaded)
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [9]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [10]:
def sentiment_score(text):
  # Create a Vader SentimentIntensityAnalyzer object
  analyzer = SentimentIntensityAnalyzer()
  # Get sentiment scores (compound score for overall sentiment)
  sentiment = analyzer.polarity_scores(text)
  return sentiment["compound"]

def named_entities(text):
  # Create a spaCy document
  doc = nlp(text)
  # Extract named entities and their labels (PERSON, ORG, etc.)
  entities = [(entity.text, entity.label_) for entity in doc.ents]
  return entities

In [11]:
df["sentiment"] = df['Quote'].apply(sentiment_score)
df["named_entities"] = df['Quote'].apply(lambda text: ", ".join([str(entity) for entity in named_entities(text)]))

In [12]:
# getting count of named_entities
df['named_entities_count'] = df['Quote'].apply(lambda x: len(named_entities(x)))

In [13]:
df.head()

,Politician,Quote,Image URL,char_count,word_count,word_length,adv_count,adj_count,noun_count,verb_count,det_count,sentiment,named_entities,named_entities_count
0,Melissa Agard,potato chips kitkat bars and viagra are not t...,true,102,16,5.312500,12.0,12.0,12.0,12.0,12.0,0.0000,"('wisconsin', 'GPE')",1
1,Melissa Agard,since the state senate has only rejected...,true,123,19,5.157895,15.0,15.0,15.0,15.0,15.0,-0.4588,"('the state senate', 'ORG'), ('five', 'CARDINA...",4
2,Instagram posts,lego donates model mri kits to hospitals to he...,true,107,17,5.352941,12.0,12.0,12.0,12.0,12.0,0.2500,,0
3,Brian Krassenstein,it has been u s policy for at least years...,true,135,25,4.200000,17.0,17.0,17.0,17.0,17.0,0.2263,"('at least years', 'DATE'), ('americans', '...",2
4,Mike Oliverio,a few years back mitchell stadium in bluefie...,true,114,20,4.450000,18.0,18.0,18.0,18.0,18.0,0.6369,"('a few years', 'DATE'), ('america', 'GPE')",2


In [14]:
df.to_csv("Data/politifact_updated.csv",index=False)